In [1]:
import sys, os
import numpy as np
import pandas as pd

# Visualization libraries
from pyfonts import load_font
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
plt.rcParams['figure.figsize'] = 12,8
palette = sns.color_palette("viridis", as_cmap=True)
sns.set_palette("viridis")

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Python version: {}". format(sys.version))
print("NumPy version: {}". format(np.__version__))
print("pandas version: {}". format(pd.__version__))
print("matplotlib version: {}". format(mpl.__version__))

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.12.10 (main, Apr 17 2025, 03:50:21) [Clang 17.0.0 (clang-1700.0.13.3)]
NumPy version: 2.2.5
pandas version: 2.2.3
matplotlib version: 3.10.1
-------------------------


### To replicate
1. Change the value of `DATA_FOLDER` to the location you will download the sources to.
2. Download each source independently using the `File used:` and other information mentioned in this notebook and dataset.
3. Run this notebook to process each individual source and compile into one CSV.

In [2]:
DATA_FOLDER = '../../data/distributions/check'

### Normal Distribution

SOCR height and weight dataset from University of Michigan

- Source: https://wiki.socr.umich.edu/index.php/SOCR_Data_Dinov_020108_HeightsWeights
- Download: https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset
- File used: SOCR-HeightWeight.csv

In [3]:
df = pd.read_csv(os.path.join(DATA_FOLDER, 'SOCR-HeightWeight.csv'))

df.rename(columns={'Height(Inches)': 'norm_height_inch', 'Weight(Pounds)': 'norm_weight_lbs'}, inplace=True)
df = df[['norm_height_inch', 'norm_weight_lbs']]

df = df[:25000]

df.to_csv(os.path.join(DATA_FOLDER, 'normal_distribution_data.csv'), index=False)

### Uniform Distribution

Lottery Take 5 Winning Number dataset from Data.gov

- Source: https://catalog.data.gov/dataset/lottery-take-5-winning-numbers
- Download: https://data.ny.gov/api/views/dg63-4siq/rows.csv?accessType=DOWNLOAD
- Terms of Use: https://data.ny.gov/download/77gx-ii52/application/pdf

In [4]:
lottery_take_5_df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'Lottery_Take_5_Winning_Numbers__Beginning_1992.csv')
)


# Function to transform a single row into multiple rows for white balls
def transform_row(row):
    draw_date = row['Draw Date']
    winning_numbers = row['Evening Winning Numbers'].strip().split()
    transformed_data = []
    for i, number in enumerate(winning_numbers):
        transformed_data.append({
            'Draw_Date': draw_date,
            'White_Ball': int(number),
            'Draw_Position': i + 1
        })
    return transformed_data


# Apply the transformation to each row of the DataFrame
transformed_list = lottery_take_5_df.apply(transform_row, axis=1).tolist()

# Flatten the list of lists into a single list of dictionaries
flat_transformed_list = [
    item for sublist in transformed_list for item in sublist]

# Create a new DataFrame from the flattened list
uniform_distribution_df = pd.DataFrame(flat_transformed_list)

# uniform_distribution_df.rename(
#     columns={
#         'Draw_Date': 'lt5_draw_date', 'White_Ball': 'lt5_winning_number',
#         'Draw_Position': 'lt5_draw_position',
#     }, inplace=True)
uniform_distribution_df.rename(
    columns={
        'Draw_Date': 'uniform_draw_date', 'White_Ball': 'uniform_winning_number',
        'Draw_Position': 'uniform_draw_position',
    }, inplace=True)

# Display the resulting DataFrame
print(uniform_distribution_df)

uniform_distribution_df = uniform_distribution_df[:25000]

# Save this DataFrame to a CSV file
uniform_distribution_df.to_csv(
    os.path.join(DATA_FOLDER, 'uniform_distribution_data.csv'),
    index=False
)

      uniform_draw_date  uniform_winning_number  uniform_draw_position
0            09/24/2020                       2                      1
1            09/24/2020                       5                      2
2            09/24/2020                      10                      3
3            09/24/2020                      15                      4
4            09/24/2020                      18                      5
...                 ...                     ...                    ...
51115        04/21/2025                      13                      1
51116        04/21/2025                      26                      2
51117        04/21/2025                      29                      3
51118        04/21/2025                      30                      4
51119        04/21/2025                      37                      5

[51120 rows x 3 columns]


### Exponential Distribution

The Ultimate Earthquake Dataset From 1990-2023

- Source: https://www.kaggle.com/datasets/alessandrolobello/the-ultimate-earthquake-dataset-from-1990-2023
- License: [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/)

In [5]:
earthquake_df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'Eartquakes-1990-2023.csv')
)

earthquake_df = earthquake_df.loc[earthquake_df['state'] == ' Japan']
print(earthquake_df.head())

# Convert the 'time' column to datetime objects
earthquake_df['time'] = pd.to_datetime(earthquake_df['time'], unit='ms')

# Sort the DataFrame by time
earthquake_df = earthquake_df.sort_values(by='time')

# Calculate the time difference between consecutive earthquakes in seconds
time_diff = earthquake_df['time'].diff().dt.total_seconds()

# Create the 'Time_Since_Last_Earthquake_Seconds' column
earthquake_df['Time_Since_Last_Earthquake_Seconds'] = time_diff

# Create a sequential 'Earthquake_Number' column as the dependent variable
earthquake_df['Earthquake_Number'] = range(1, len(earthquake_df) + 1)

earthquake_df['Magnitude'] = earthquake_df['magnitudo']

# Select the two columns we need for the exponential distribution example
exponential_df = earthquake_df[[
    'Earthquake_Number', 'Time_Since_Last_Earthquake_Seconds', 'Magnitude']].dropna()

exponential_df.rename(
    columns={
        'Earthquake_Number': 'exp_earthquake_number', 'Time_Since_Last_Earthquake_Seconds': 'exp_time_since_last_earthquake_seconds',
        'Magnitude': 'exp_magnitude',
    }, inplace=True)

# Display the first few rows of the resulting DataFrame
print(exponential_df.head())

exponential_df = exponential_df[:25000]

# Save this DataFrame to a new CSV file
exponential_df.to_csv(
    os.path.join(
        DATA_FOLDER, 'exponential_distribution_data.csv'),
    index=False
)

             time                           place    status  tsunami  \
66   631184592880  3 km NE of Hitachi-Naka, Japan  reviewed        0   
245  631297597730      40 km ENE of Nemuro, Japan  reviewed        0   
346  631356818930       50 km SE of Hasaki, Japan  reviewed        0   
480  631455398380     73 km ESE of Hitachi, Japan  reviewed        0   
544  631495557190      252 km SSE of ?yama, Japan  reviewed        0   

     significance   data_type  magnitudo   state  longitude  latitude  depth  \
66            354  earthquake        4.8   Japan    140.568    36.417   67.9   
245           326  earthquake        4.6   Japan    146.063    43.420   73.4   
346           284  earthquake        4.3   Japan    141.272    35.457   10.0   
480           298  earthquake        4.4   Japan    141.408    36.342   55.1   
544           416  earthquake        5.2   Japan    138.821    32.381  247.7   

                                 date  
66   1990-01-01 09:03:12.880000+00:00  
245  1

### Poisson Distribution

Schochastics's Football (Soccer) Dataset

- Source: https://github.com/schochastics/football-data
- Download: https://github.com/schochastics/football-data/archive/refs/tags/v0.1.0.zip
- File used: football-data-0.1.0/data/results/spain.csv
- License: [Open Data Commons Attribution License](https://opendatacommons.org/licenses/by/1-0/index.html)

In [6]:
df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'spain.csv')
)

df = df[['home', 'away', 'date', 'gh', 'ga']]

# df.rename(columns={'home': 'home_team', 'away': 'away_team', 'gh': 'goals_home', 'ga': 'goals_away'}, inplace=True)
df.rename(columns={
    'home': 'poisson_home_team', 'away': 'poisson_away_team',
    'gh': 'poisson_goals_home', 'ga': 'poisson_goals_away',
    'date': 'poisson_date'
}, inplace=True)

df = df[:25000]

df.to_csv(
    os.path.join(
        DATA_FOLDER, 'poisson_distribution_data.csv'),
    index=False
)

### Log Normal Distribution

DC Public Employee Salary Dataset

- Source 1: https://catalog.data.gov/dataset/dc-public-employee-salary 
- Source 2: https://opendata.dc.gov/datasets/DCGIS::dc-public-employee-salary/explore
- Download: https://opendata.dc.gov/api/download/v1/items/c9a03cab565b44849bcfc57e63fd3591/csv?layers=35
- License: [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0)

In [7]:
df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'DC_Public_Employee_Salary.csv')
)

df.dropna(inplace=True)
df = df.loc[df['COMPRATE'] > 5000]

df['employee_name'] = df['FIRST_NAME'] + ' ' + df['LAST_NAME']

print(df.head())

df = df[['employee_name', 'JOBTITLE', 'COMPRATE']]
# df.rename(columns={'COMPRATE': 'annual_compensation', 'JOBTITLE': 'job_title'}, inplace=True)
df.rename(columns={
    'employee_name': 'ln_employee_name', 'COMPRATE': 'ln_annual_compensation', 'JOBTITLE': 'ln_job_title'
}, inplace=True)

df = df[:25000]

df.to_csv(
    os.path.join(
        DATA_FOLDER, 'log_normal_distribution_data.csv'),
    index=False
)

  FIRST_NAME  LAST_NAME                       JOBTITLE DESCRSHORT GRADE  \
0      Scott     Pitzer  Facilities Systems Specialist        DBH    12   
1    Pauline     Oboite                          NURSE        DBH    05   
2       Dawn       Fong   Special Projects Coordinator        DCG    12   
3      Sonya  Kingsland   Behavioral Health Technician        DBH    08   
4    Octavia   Fletcher   Behavioral Health Technician        DBH    08   

   COMPRATE HIREDATE_STRING GVT_TYPE_OF_APPT  OBJECTID     employee_name  
0   90805.0      1987/10/01         CS - Reg         1      Scott Pitzer  
1   76510.0      2000/02/14         CS - Reg         2    Pauline Oboite  
2   90805.0      2020/03/16        CS - Term         3         Dawn Fong  
3   70600.0      1988/06/07         CS - Reg         4   Sonya Kingsland  
4   70600.0      1995/09/18         CS - Reg         5  Octavia Fletcher  


### Gamma Distribution

MQTT Traffic Dataset

- Source: https://www.kaggle.com/datasets/cnrieiit/mqttset
- Download: `mqttdataset_reduced.csv` within `Data/FINAL_CSV/`
- License: [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/)
- Attribution: [Vaccari, I.; Chiola, G.; Aiello, M.; Mongelli, M.; Cambiaso, E. MQTTset, a New Dataset for Machine Learning Techniques on MQTT. Sensors 2020, 20, 6578](https://www.mdpi.com/1424-8220/20/22/6578/htm)

In [8]:
df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'mqttdataset_reduced.csv')
)

df = df.loc[df['tcp.len'] > 0]

df = df[['tcp.time_delta', 'tcp.len']]

df.rename(columns={
    'tcp.time_delta': 'gamma_time_delta', 'tcp.len': 'gamma_data_length'
}, inplace=True)

df = df[:25000]

df.to_csv(
    os.path.join(
        DATA_FOLDER, 'gamma_distribution_data.csv'),
    index=False
)

### Beta Distribution

College Completion Dataset

- Source: https://www.kaggle.com/datasets/thedevastator/boost-student-success-with-college-completion-da?select=cc_state_sector_grads.csv
- Download: `cc_state_sector_grads.csv`
- License: [CC0 1.0 Universal (CC0 1.0)](https://creativecommons.org/publicdomain/zero/1.0/)
- Attribution: https://data.world/databeats/college-completion


In [9]:
df = pd.read_csv(
    os.path.join(
        DATA_FOLDER, 'cc_state_sector_grads.csv')
)

df.rename(
    columns={
        'state': 'cc_state', 'year': 'cc_year', 'gender': 'cc_gender',
        'grad_100_rate': 'cc_completion_100_rate'
    }, inplace=True)


df = df.loc[df['cc_state'].isin(
    [
        'California', 'Texas', 'Florida', 'New York', 'Pennsylvania',
        'Illinois', 'Ohio', 'Georgia', 'North Carolina', 'Michigan',
        'New Jersey', 'Virginia', 'Washington', 'Arizona', 'Tennessee'
    ]
)]
df = df[['cc_state', 'cc_year', 'cc_gender', 'cc_completion_100_rate']]

df['cc_completion_100_rate'] = df['cc_completion_100_rate'] / 100

df.rename(
    columns={
        'cc_state': 'beta_state', 'cc_year': 'beta_year', 'cc_gender': 'beta_gender',
        'cc_completion_100_rate': 'beta_completion_100_rate'
    }, inplace=True)

print(len(df))
df = df[:25000]

print(df.head())

df.to_csv(
    os.path.join(
        DATA_FOLDER, 'beta_distribution_data.csv'),
    index=False
)

26694
    beta_state  beta_year beta_gender  beta_completion_100_rate
324    Arizona       2011           B                     0.094
325    Arizona       2011           B                     0.170
326    Arizona       2011           B                     0.072
327    Arizona       2011           B                     0.111
328    Arizona       2011           B                     0.131


In [10]:
normal_df = pd.read_csv(os.path.join(DATA_FOLDER, 'normal_distribution_data.csv'))
uniform_df = pd.read_csv(os.path.join(DATA_FOLDER, 'uniform_distribution_data.csv'))
exponential_df = pd.read_csv(os.path.join(DATA_FOLDER, 'exponential_distribution_data.csv'))
poisson_df = pd.read_csv(os.path.join(DATA_FOLDER, 'poisson_distribution_data.csv'))
ln_df = pd.read_csv(os.path.join(DATA_FOLDER, 'log_normal_distribution_data.csv'))
gamma_df = pd.read_csv(os.path.join(DATA_FOLDER, 'gamma_distribution_data.csv'))
beta_df = pd.read_csv(os.path.join(DATA_FOLDER, 'beta_distribution_data.csv'))

final_df = pd.concat(
    [
        normal_df, uniform_df, exponential_df, poisson_df, ln_df, gamma_df, beta_df
    ], axis=1
)

final_df.to_csv(os.path.join(DATA_FOLDER, 'real_world_distributions.csv'), index=False)